## This notebook uses the parenthood.csv file available in Brightspace /Content/Data/

# Linear regression

This chapter is adapted from Danielle Navarro's excellent [Learning Statistics with R](https://learningstatisticswithr.com) book. Minor changes to the text and adapting the code to Python by Todd Gureckis and Shannon Tubridy.

The goal in the first part of this notebook is to introduce **_linear regression_**, the standard tool that statisticians rely on when analysing the relationship between interval scale predictors and interval scale outcomes. 

Stripped to its bare essentials, linear regression models are basically a slightly fancier version of the Pearson correlation, though regression models are much more powerful tools. 

In the correlation and t-test notebooks we learned about the Pingouin and Scipy libraries. We will continue to make use of those and introduce a new library called statsmodels that provides some nice regression tools.

In [ ]:
import numpy.random as npr
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
import pingouin as pg
import warnings


## What is a linear regression model?

To kick things off we can look at the relationship between the amount of sleep a baby and parent get each night and the daily grumpiness of the parent.

Start with loading our data into a dataframe using pandas

In [ ]:
parenthood_df = pd.read_csv('../../../data/stats_data/parenthood.csv', sep = ',')
parenthood_df.head()

Our data contains measurements for four variables corresponding to: 

- the nightly sleep for the parent (`parent_sleep`)
- nightly sleep for the baby (`baby_sleep`)
- the parent's grumpiness level the next day (`parent_grumpy`)  
- which day of the observation period we are referring to (`day`).

Our working hypothesis is that that the amount of parent sleep each night impacts the next day's grumpiness.

The first thing we might do is plot a scatter plot to view the distribution of the data according to those two measurements.

We already know how to do scatter plots in seaborn:

In [ ]:
sns.relplot(x='parent_sleep', y='parent_grumpy', data=parenthood_df)

plt.title('Scatter plot of parent grumpiness versus parent sleep')

We can also check the correlation between these two variables using pingouin (pg) corr function:

In [ ]:
corr_results = pg.corr(x=parenthood_df['parent_sleep'], 
        y=parenthood_df['parent_grumpy'])

corr_results

In [ ]:
# or accessing the values by indexing
corr_results['p-val'][0]

In [ ]:
# get the correlation statistic (r) as well
rval = corr_results['r'].values[0]
rval

In [ ]:
# Correlation with scipy
scipy_corr = stats.pearsonr(parenthood_df['parent_sleep'], 
                            parenthood_df['parent_grumpy'])
# r value:
scipy_corr[0]


# p value:
scipy_corr[1]

The correlation notebook from a previous meeting has more info on running and interpreting correlations, but we can note here that the r value is -.9 suggesting a strong negative relationship between amount of parent sleep and next day grumpiness.

The scatterplot and correlation test are promising but we are proposing not just a relationship between the variables but that one leads to the other: getting more (or less) sleep each night has a causal link to grumpiness.

We can't conclusively determine causality given our data, but we would like to do an analysis that allows us to assess the relationship between these two variables in a way that gives us a prediction of how much grumpiness we should observe each day given some known amount of sleep the night before.

We want something like this next figure: we want a straight line through the middle of the data that can relate any X value (sleep) to an expected Y value (grumpinness).

This line is called a **_regression line_** and we can quickly vizualize it using seaborn regplot():

In [ ]:
sns.regplot(x='parent_sleep', 
            y='parent_grumpy', 
            ci=None, 
            data=parenthood_df)

plt.title('Scatter plot of parent sleep and parent grumpiness including best fit regression line')

Linear regression gives us a way of finding the line that best fits the data, with a very specific meaning of "fits the data"

Let's take a quick detour to a refresher of some basic math making lines.

The formula for a straight line is usually written like this:

$
y = mx + c
$ 


The two *variables* are $x$ and $y$, and we have two *coefficients*, $m$ and $c$. The coefficient $m$ represents the *slope* of the line, and the coefficient $c$ represents the *$y$-intercept* of the line. 

The intercept is interpreted as "the value of $y$ that you get when $x=0$". 

A slope of $m$ means that if you increase the $x$-value by 1 unit, then the $y$-value goes up by $m$ units; a negative slope means that the $y$-value would go down rather than up.

These ideas map directly onto our proposed analysis of the sleep data (how much does grumpiness change given some change in parent sleep?), and this equation for a line is the exame same formula for describing a regression line. If $Y$ is the outcome variable (the DV) and $X$ is the predictor variable (the IV), then the formula that describes our regression is written like this:

$
\hat{Y_i} = b_1 X_i + b_0
$


Looks like the same formula, but there's some extra bits in this version.

1) We have $X_i$ and $Y_i$ rather than just plain old $X$ and $Y$. This is because we want to remember that we're dealing with actual data. In this equation, $X_i$ is the value of predictor variable for the $i$th observation (i.e., the number of hours of sleep on day $i$ of the study), and $Y_i$ is the corresponding value of the outcome variable (i.e., grumpiness on that day). In multiple regression we can have multiple predictors so we would have i = 1:n. predictors

2) Notice that it's now $\hat{Y}_i$ and not $Y_i$. This is because we want to make the distinction between the *actual data* $Y_i$, and the *estimate* $\hat{Y}_i$ (i.e., the prediction that our regression line is making). 

Thirdly, I changed the letters used to describe the coefficients from $m$ and $c$ to $b_1$ and $b_0$. That's just the way that statisticians like to refer to the coefficients in a regression model. I've no idea why they chose $b$, but that's what they did. In any case $b_0$ always refers to the intercept term, and $b_1$ refers to the slope in a single predictor regression.

Here is the plot again:

In [ ]:
sns.regplot(x='parent_sleep', 
            y='parent_grumpy', 
            ci=None, 
            data=parenthood_df)

plt.title('Scatter plot of parent sleep and parent grumpiness including best fit regression line')

One clear thing is that the data don't fall perfectly on the line: the data $Y_i$ are not identical to the predictions of the regression model $\hat{Y_i}$. 

The difference between a model prediction for parent_grumpy given some input parent_sleep value and the actual value of parent_grumpy for that parent_sleep value is known as a *residual*, and we'll refer to it as $\epsilon_i$.  Written using mathematics, the residuals are defined as:

$
\epsilon_i = Y_i - \hat{Y}_i
$


which in turn means that we can write down the complete linear regression model as:

$
Y_i = b_1 X_i + b_0 + \epsilon_i
$

The $\epsilon$ symbol is the Greek letter epsilon. It's traditional to use $\epsilon_i$ or $e_i$ to denote a residual.

## Estimating the linear regression model

Here is our data again, but now showing the size of the residuals for each datapoint. This next cell fits a regression model but we'll come back to the details of that in a little bit.


In [ ]:
# fit a linear regression to the grumpy column given parent sleep
lr = smf.ols(formula="parent_grumpy ~ parent_sleep", data=parenthood_df)
fit = lr.fit() # fit
# params, pred, resid = fit.params, fit.fittedvalues, fit.resid
params = fit.params
pred = fit.fittedvalues
resid = fit.resid

# setup a figure and get handles to fig and axes
fig, ax = plt.subplots(figsize=(9,6))

# set the x values to be evenly spaced values from min to max of parent sleep
x1 = np.linspace(parenthood_df['parent_sleep'].min(), parenthood_df['parent_sleep'].max(), 400)

# get y values by pushing all the x values through the regression line:
y1 = params['parent_sleep']*x1+params['Intercept']

# plot the actual data as dots
ax.plot(parenthood_df['parent_sleep'], 
        parenthood_df['parent_grumpy'],
        'ko',markersize=4)

# plot each estimated data point Y_i as a white dot
ax.plot(parenthood_df['parent_sleep'], 
        parenthood_df['parent_grumpy']-resid,
        'o',
        markersize=4,
        markeredgecolor='r', 
        markeredgewidth=.4, 
        markerfacecolor='white')

# plot the regression line:
ax.plot(x1,y1,'-',color='steelblue',linewidth=1)

# make vertical lines showing the residual error for each
# estimated vs true Y value:
ax.vlines(parenthood_df['parent_sleep'], 
          parenthood_df['parent_grumpy'], 
          parenthood_df['parent_grumpy']-resid,
          'r',
          linewidth=0.5)

# label the y and y axis
plt.xlabel("sleep (hours)")
plt.ylabel("grumpiness (0-100)")



Here we see the best fitting regression line along with the residuals. This line is the best fit because it's the one that, when drawn through the data, has the smallest squared residuals. More formally:


> The estimated regression coefficients, $\hat{b}_0$ and $\hat{b}_1$ are those that minimise the sum of the squared residuals, which we could either write as $\sum_i (Y_i - \hat{Y}_i)^2$ or as $\sum_i {\epsilon_i}^2$.

Note: there is actually more than one way to estimate a regression model, so the more technical name for the estimation process described here is **_ordinary least squares (OLS) regression_**.  


Now have a concrete definition for what counts as our "best" choice of regression coefficients, $\hat{b}_0$ and $\hat{b}_1$. The natural question now is how do we *find* these numbers. 

The full answer to this question is complicated, and it doesn't help you understand the logic of regression. As a result, let's cut straight to the chase and use the [statsmodels](https://www.statsmodels.org/stable/index.html) `ols()` function (short for "ordinary least squares") to do all the heavy lifting. 

### Using the `statsmodels ols()` function

There are a number of libraries that will perform linear regression in python.  These range from complex machine learning libraries (e.g., [scikit-learn](https://scikit-learn.org/stable/)) to simple statistics libraries (e.g., [pingouin](https://pingouin-stats.org)).  Here we will focus on the [statsmodels](https://www.statsmodels.org/stable/index.html) library because it is one that is most geared toward traditional statistical analysis.

The ordinary least squares algorithm is accessible via the `statsmodel.smf` submodule.  To import it do this:

In [ ]:
import statsmodels.formula.api as smf

This will provide access to a function `ols()` which will perform ordinary least squares regression. 

The `ols()` function is a fairly complicated one: if you imported the library using the command above and type `smf.ols?`, the help files will reveal that there are a lot of arguments that you can specify. At this stage however, there's really only two of them that we care about:


- `formula`. A formula that specifies the regression model. For the simple linear regression models that we've talked about so far, in which you have a single predictor variable as well as an intercept term, this formula is of the form `outcome ~ predictor`. However, more complicated formulas are allowed, and we'll discuss them later.
- `data`. The data frame containing the variables.

The output of the `smf.ols()` function is a fairly complicated object, with quite a lot of technical information buried under the hood. Because this technical information is used by other functions, it's generally a good idea to create a variable that stores the results of your regression. To run a single variable linear regression, the command is this:

In [ ]:
# define the linear model
lr_model = smf.ols(formula="parent_grumpy ~ parent_sleep", data=parenthood_df)
# fit the model (get the best coefficients to minimize the residuals)
lr =  lr_model.fit()


#### Note that if you specify the regression model using the formula= syntax then the OLS function will automatically add a constant predictor to your model (which you want)

In the ols() call we defined our regression model and specified where the data are. 

The model is described in the formula "parent_grumpy ~ parent_sleep". 

This syntax is saying model parent_grumpy as a function of, or based on the values in, parent_sleep. `parent_grumpy` is the outcome variable and `parent_sleep` is the predictor variable. 

Later we will see multiple regression using more than one predictor.

That defined the model, and then to estimate the best fitting parameters we called the .fit() method, or function, attached to our defined model.

We could also do it in one line by _chaining_ the .fit() command onto the end of the sms.ols() call:

In [ ]:
lr = smf.ols(formula="parent_grumpy ~ parent_sleep", data=parenthood_df).fit()


We can look at the regression results stored in `lr` after we .fit() the model.

Lets focus on `lr.params` for a moment.

In [ ]:
lr.params


There are two separate pieces of information here. First python gives us the intercept $\hat{b}_0 = 125.96$ and then the slope or coefficient associated with any predictor variables. In our case that is just the sleep variable $\hat{b}_1 = -8.94$. 

In other words, the best-fitting regression line that I plotted earlier has this formula:

$
\hat{Y}_i = -8.94 \ X_i + 125.96
$


We can access them separately like this:

In [ ]:
lr.params['parent_sleep']

In [ ]:
lr.params['Intercept']

### Interpreting the estimated model

Let's start with $\hat{b}_1$, the slope. 

If we remember the definition of the slope, a regression coefficient of $\hat{b}_1 = -8.94$ means that if I increase $X_i$ by 1, then I'm decreasing $Y_i$ by 8.94: each additional hour of sleep will improve parent mood, reducing grumpiness by 8.94 grumpiness points. 

What about the intercept? Since $\hat{b}_0$ corresponds to "the expected value of $Y_i$ when $X_i$ equals 0", it's pretty straightforward. It implies that if this person gets zero hours of sleep ($X_i =0$) then grumpiness will go off the scale, to an insane value of ($Y_i = 125.96$). Very grumpy.

We can use the results output to manually plot the best fitting line:

In [ ]:
# use seaborn to scatterplot the original data:
sns.relplot(x='parent_sleep', 
            y = 'parent_grumpy', 
            data=parenthood_df)

# use numpy to make 20 evenly space values from 0 to 9
x = np.linspace(-20, 9, 20)

# use the regression parameters to get estimated y values for each input x
y = lr.params['parent_sleep']*x + lr.params['Intercept']

# add the line to the scatter plot
plt.plot(x, y)


#### Notice that the regression model gives us a way to predict parent_grumpy levels for values of parent_sleep that have not actually been observed yet. This is a very powerful aspect of regression-based analysis.

In [ ]:
# How grumpy would someone who get 2.7 hours of sleep be?
x = 2.7

# push an x value through the best fit line equation to get predicted y
y = lr.params['parent_sleep']*x + lr.params['Intercept']
y

The OLS regression results that we stored in the variable `lr` has lots more information available to us and we can get a comprehensive output using the lr.summary() command. Later in this notebook we'll come back parse the output in more detail.

In [ ]:
lr.summary()

## Multiple linear regression

The simple linear regression model that we've discussed up to this point assumes that there's a single predictor variable that you're interested in, in this case `parenthood_df['parent_sleep']`. 

In many research projects you actually have multiple predictors that you want to examine and we can do this with **_multiple regression_**.

Multiple regression is conceptually very simple. All we do is add more terms to our regression equation. 

Suppose that we've got two variables that we're interested in; perhaps we want to use both `parent_sleep` and `baby_sleep` to predict the `parent_grumpy` variable. 

As before, we let $Y_i$ refer to grumpiness on the $i$-th day. 

But now we have two $X$ variables: the first corresponding to the amount of sleep the parent got and the second corresponding to the amount of sleep the baby got. 

So we'll let $X_{i1}$ refer to the hours the parent slept on the $i$-th day, and $X_{i2}$ refers to the hours that the baby slept on that day. Now we can write our regression model like this:

$
Y_i = b_2 X_{i2} + b_1 X_{i1} + b_0 + \epsilon_i
$

As before, $\epsilon_i$ is the residual associated with the $i$-th observation, $\epsilon_i = {Y}_i - \hat{Y}_i$. 

In this model, there are three coefficients to be estimated: $b_0$ is the intercept, $b_1$ is the coefficient associated with parent sleep, and $b_2$ is the coefficient associated with baby sleep. 

Although the number of coefficients that need to be estimated has changed, the basic idea of how the estimation works is unchanged: our estimated coefficients $\hat{b}_0$, $\hat{b}_1$ and $\hat{b}_2$ are those that minimise the sum squared residuals. 

### Doing it in Python

Multiple regression in python is no different to simple regression: all we have to do is specify a more complicated `formula` when using the `smf.ols()` function. For example, if we want to use both `parent_sleep` and `baby_sleep` as predictors in our attempt to explain grumpiness, then the formula we need is this:
```
                    parent_grumpy ~ parent_sleep + baby_sleep
```
Notice that, just like last time, I haven't explicitly included any reference to the intercept term in this formula; only the two predictor variables and the outcome. 

Now we'll create a new regression model -- `lr2` -- using the following command:

In [ ]:
lr2_model = smf.ols(formula = "parent_grumpy ~ parent_sleep + baby_sleep", 
                    data = parenthood_df)
lr2 = lr2_model.fit()

And just like last time, if we look at the `fit.params` from this regression model we can see what the estimated regression coefficients are.

In [ ]:
lr2.params

In [ ]:
# get the predictor coefficient from the results object lr
b1 = lr2.params['parent_sleep']

b2 = lr2.params['baby_sleep']

# get the estimated intercept from the results object lr
Intercept = lr2.params['Intercept']


X_1 = 3
X_2 = 10

# make estimated Y values by 
# multiplying the coefficient times the actual parent 
# sleep levels in X and add the intercept
Y_pred = b1*X_1 + b2*X_2 + Intercept
Y_pred

The coefficient associated with `parent_sleep` is quite large, suggesting that every hour of parent sleep lost makes them a lot grumpier. However, the coefficient for `baby_sleep` is very small, suggesting that it doesn't really matter how much sleep the baby gets.

### Formula for the general case

The equation that I gave above shows you what a multiple regression model looks like when you include two predictors. if you want more than two predictors all you have to do is add more $X$ terms and more $b$ coefficients. In other words, if you have $K$ predictor variables in the model then the regression equation looks like this:

$
Y_i = \left( \sum_{k=1}^K b_{k} X_{ik} \right) + b_0 + \epsilon_i
$



### Interpreting coefficients in multiple regression

The difference between regular ("simple") regression and multiple regression really just boils down to the number of predictors you have in your formula.  

However, the coefficients also have a little different meaning.  In the single regression case we found that the coefficient associated with the `parent_sleep` regressor -8.936.  This we interpreteted as meaning that for every additional hour of sleep that the parent gets, grumipness goes down by about 9 points.  

The interpretation of the coefficient on the `parent_sleep` regressor in the multiple linear regression case (`lr2` above) is a little different.  

One common interpretation of the multiple regression coefficients is the change in the outcome variable ("grumpiness") for a corresponding change in the predictor `sleep` **when all other predictors are held constant**.  That last part in bold is an important additional caveat.  It kind of makes sense because if you set the value of all other predictors to zero then the multiple regression becomes the same as a simple regression with a single predictor.  

Another way to say this is that the coefficient $b_k$ is called the *partial regression coefficient*.  This is because it represents the contribution of predictor $X_k$ after it has been adjusted for the other predictors. 

## Quantifying the fit of the regression model

So far we have estimated the coefficients of a linear regression model.

Remember, the regression model only produces a prediction $\hat{Y}_i$ about what parent mood is like: the actual mood is $Y_i$. If these two are very close, then the regression model has done a good job. If they are very different, then it has done a bad job.

### The $R^2$ value

Once again, let's wrap a little bit of mathematics around this. Firstly, we've got the sum of the squared residuals:

$
\mbox{SS}_{res} = \sum_i (Y_i - \hat{Y}_i)^2
$

which we would hope to be pretty small. Specifically, what we'd like is for it to be very small in comparison to the total variability in the outcome variable, 

$
\mbox{SS}_{tot} = \sum_i (Y_i - \bar{Y})^2
$

We can calculate this ourselves. To make the python commands look more similar to the mathematical equations, we can create variables `X` and `Y` from our dataframe:

In [ ]:
X = parenthood_df['parent_sleep']  # the predictor
Y = parenthood_df['parent_grumpy']  # the outcome

Now that we've done this, let's calculate the $\hat{Y}$ values and store them in a variable called `Y_pred`. For the simple model that uses only a single predictor, we can do this by getting the slope and parent_sleep coefficient from the model and pushing our parent_sleep data through the equation for a line:

In [ ]:
lr = smf.ols('parent_grumpy ~ parent_sleep', data=parenthood_df).fit()
lr.params

In [ ]:
# get the predictor coefficient from the results object lr
b1 = lr.params['parent_sleep']
# get the estimated intercept from the results object lr
Intercept = lr.params['Intercept']

# make a set of estimated Y values by 
# multiplying the coefficient times the actual parent 
# sleep levels in X and add the intercept
Y_pred = b1*X + Intercept

`Y_pred` stores the regression model predictions for grumpyiness any given day based on the parent_sleep value for that day.

In [ ]:
Y_pred

Now we can calculate our sum of squared residuals. We would do that using the following command:

In [ ]:
# the numpy sum() function returns the sum of
# all the values that are inside of it
# So, we are getting the the difference between
# all the true parent grumpy levels stored in Y
# and the estimated grumpy levels we predict from our
# regression model, and then square the difference
SS_resid = np.sum( (Y - Y_pred)**2 )
print( SS_resid )

The previous line took the difference between each Y and each y_pred, and then squared it (**2) and then used the np.sum() function to take the sum over all of the resulting values. This gave us a number that doesn't mean much yet.

Next we calculate the total sum of squares. The total sum of squares is the difference of each Y value from the mean of the Y values, squared, and then summed. Here's the code:

In [ ]:
SS_tot = np.sum( (Y - np.mean(Y))**2 )
print( SS_tot )

Like the SS_resid this isn't very interpretable as is, although note that our goal is to build a model of the data that accounts for as much of the variability in the outcome predictor as possible. This means that we would like the SS_resid to be as close to zero as possible, and more generally for it to be smaller than SS_tot. 

If SS_tot - SS_resid is zero it means we accounted for *all* of the variability in the data. 

If SS_tot - SS_resid is the same as SS_tot it means we accounted for *none* of the varability in the data. 

Usually we'll be somewhere in between. 

We are building towards a summary of this comparison called $R^2$ which is a standardized nummber that is equal to 1 if the regression model makes no errors in predicting the data. If the model is completely useless, $R^2$ will be equal to 0. 

The formula that provides us with the $R^2$ value is pretty simple to write down,

$
R^2 = 1 - \frac{\mbox{SS}_{res}}{\mbox{SS}_{tot}}
$

and equally simple to calculate in python:

In [ ]:
R_squared = 1.0 - (SS_resid / SS_tot)
print( R_squared )

The $R^2$ value, sometimes called the **_coefficient of determination_** has a simple interpretation: it is the *proportion* of the variance in the outcome variable that can be accounted for by the predictor(s). So in this case, the fact that we have obtained $R^2 = .816$ means that the predictor (`sleep`) explains 81.6\% of the variance in the outcome (`grump`). 

Naturally, you don't actually need to type in all these commands yourself if you want to obtain the $R^2$ value for your regression model. All you need to do is use the `lr.summary()` function. 

In [ ]:
lr.summary()

Or to get the value directly you can access it like this:

In [ ]:
lr.rsquared

In [ ]:
# round to 3 decimal places
np.round(lr.rsquared, 3)

### The relationship between regression and correlation

Regression, in this simple form that we've discussed so far, is basically the same thing as a correlation. Previously, we used the symbol $r$ to denote a Pearson correlation and the squared correlation $r^2$ is identical to the $R^2$ value for a linear regression with only a single predictor. To illustrate this, here's the squared correlation:

In [ ]:
corr_results = stats.pearsonr(X, Y)
corr_results

And squaring pearson's r gives us $R^2$:

In [ ]:
corr_results[0]**2

In other words, running a Pearson correlation is more or less equivalent to running a linear regression model that uses only one predictor variable, although a regression model gives you the coefficients and intercept that enable prediction of new values.

### The adjusted $R^2$ value

Sometimes people report a slightly different measure of model performance, known as "adjusted $R^2$". The motivation behind calculating the adjusted $R^2$ value is the observation that adding more predictors into the model will *always* cause the $R^2$ value to increase (or at least not decrease). The adjusted $R^2$ value introduces a slight change to the calculation. For a regression model with $K$ predictors, fit to a data set containing $N$ observations, the adjusted $R^2$ is:

$
\mbox{adj. } R^2 = 1 - \left(\frac{\mbox{SS}_{res}}{\mbox{SS}_{tot}} \times \frac{N-1}{N-K-1} \right)
$

This adjustment is an attempt to take the degrees of freedom into account. The big advantage of the adjusted $R^2$ value is that when you add more predictors to the model, the adjusted $R^2$ value will only increase if the new variables improve the model performance more than you'd expect by chance. The big disadvantage is that the adjusted $R^2$ value *can't* be interpreted in the elegant way that $R^2$ can. $R^2$ has a simple interpretation as the proportion of variance in the outcome variable that is explained by the regression model but there isn't an equivalent interpretation for adjusted $R^2$. 

## Hypothesis tests for regression models

So far we've talked about what a regression model is, how the coefficients are esimated, and how to quantify the peformance of the model.

Next is hypothesis tests. 

There are two different (but related) kinds of hypothesis tests we'll discuss: 

1) testing whether the regression model as a whole is performing significantly better than a null model at accounting for the data

2) testing whether a particular regression coefficient is significantly different from zero. 

### Testing the model as a whole

The first hypothesis we'll discuss is one in which the null hypothesis that there is *no relationship* between the predictors and the outcome, and the alternative hypothesis is that *the data are distributed in exactly the way that the regression model predicts*. 

Formally, our "null model" corresponds to the fairly trivial "regression" model in which we include 0 predictors, and only include the intercept term $b_0$

$
H_0: Y_i = b_0 + \epsilon_i
$

If our regression model has $K$ predictors, the "alternative model" is described using the usual formula for a multiple regression model:

$
H_1: Y_i = \left( \sum_{k=1}^K b_{k} X_{ik} \right) + b_0 + \epsilon_i
$

To test these hypotheses against each other we need to understand that it's possible to divide up the total variance $\mbox{SS}_{tot}$ into the sum of the residual variance 
$\mbox{SS}_{res}$ and the regression model variance $\mbox{SS}_{mod}$. 

$
\mbox{SS}_{mod} = \mbox{SS}_{tot} - \mbox{SS}_{res}
$


We convert these sums of squares into mean squares by dividing by the degrees of freedom.

$
\begin{array}{rcl}
\mbox{MS}_{mod} &=& \displaystyle\frac{\mbox{SS}_{mod} }{df_{mod}} \\ \\
\mbox{MS}_{res} &=& \displaystyle\frac{\mbox{SS}_{res} }{df_{res} }
\end{array}
$

The $df$ associated with the model is closely tied to the number of predictors that we've included. It turns out that $df_{mod} = K$. For the residuals, the total degrees of freedom is $df_{res} = N - K - 1$. 

Now that we've got our mean square values, we can calculate an $F$-statistic like this:

$
F =  \frac{\mbox{MS}_{mod}}{\mbox{MS}_{res}}
$


and the degrees of freedom associated with this are $K$ and $N-K-1$. This $F$ statistic has exactly the same interpretation as it does in ANOVA. Large $F$ values indicate that the null hypothesis is performing poorly in comparison to the alternative hypothesis. The link between regression and ANOVA is not coincidental: they are essentially the same test with some minor differences. We'll return to ANOVA later.

### Tests for individual coefficients

The $F$-test that we've just introduced is useful for checking that the model as a whole is performing better than chance. This is important: if your regression model doesn't produce a significant result for the $F$-test then you probably don't have a very good regression model (or, quite possibly, you don't have very good data). 

However, while failing this test is a pretty strong indicator that the model has problems, *passing* the test (i.e., rejecting the null) doesn't imply that the model is good! Why is that, you might be wondering? The answer to that can be found by looking at the coefficients for the model:

In [ ]:
# estimate the regression model predicting parent_grumpy scores from a combination of parent_sleep
# and baby_sleep
lr = smf.ols(formula="parent_grumpy ~ parent_sleep + baby_sleep", data=parenthood_df).fit()
print(lr.params)

Notice that the estimated regression coefficient for the `babysleep` variable is tiny (~0.01), relative to the value that we get for `sleep` (-8.95). Given that these two variables are absolutely on the same scale (they're both measured in "hours slept"), it looks like its really only the amount of parent_sleep that matters for predicting grumpiness.

To test this we can reuse a hypothesis test that we discussed earlier, this time the $t$-test. The test that we're interested has a null hypothesis that the true regression coefficient is zero ($b = 0$), which is to be tested against the alternative hypothesis that it isn't ($b \neq 0$). That is:

$
\begin{array}{rl}
H_0: & b = 0 \\
H_1: & b \neq 0 
\end{array}
$

We test this using a t stastisic like we've previously seen, comparing the estimated coefficient versus a null hypothesis that it is equal to zero.


$
t = \frac{\hat{b}}{\mbox{SE}({\hat{b})}}
$


The degrees of freedom and error term are calculated a little differently than in the Students t-test, but this $t$-statistic can be interpreted in the same way as the $t$-statistics for the differences between groups. 

### Running the hypothesis tests in Python

To compute all of the quantities that we have talked about so far, all you need to do is ask for a `summary()` of your regression model. Let's go back to the grumpiness predictor using two variables (multiple regression):

In [ ]:
mlr = smf.ols(formula="parent_grumpy ~ parent_sleep + baby_sleep", 
              data=parenthood_df).fit()
mlr.summary()

The output that this command produces is pretty dense, but we've already discussed a few parts of interest in it. 

It is a little hard to tell from the output, but there are three tables.  

### Overall model information

First it reminds us that the dependent measure in our model is `"parent_grumpy"`.  

Next we see the **R-squared** and **Adj. R-squared** which are the two versions of the $R^2$ fit statistic that we just discussed.  

The **F-statistic** is the ratio of the variance explained by the model to the residuals. The **Prob(F-statistic)** is the probability of obtaining that value of the F-statistic (or greater) under the F-distribution implied by the degrees of freedom from the model, in this case F(2,97)...  Can you see where I read the degrees of freedom from? 


### Information on individual predictors

The second table is the table of tests on individual coefficients in the model. Each row in this table refers to one of the coefficients in the regression model. The first row is the intercept term (**Intercept**), and the later ones look at each of the predictors (**sleep** and **babysleep** in this case). The columns give you all of the relevant information. 

The first column ('coefficient') is the actual estimate of $b$ (e.g., 125.96 for the intercept, and -8.9 for the `parent_sleep` predictor). 

The second column is the standard error estimate $\hat\sigma_b$. 

The third column gives you the $t$-statistic and  the fourth column gives you the actual $p$ value (P>|t|) for each of these tests.  The final two columns show the 95% confidence intervals on the values of the parameters. The only thing that the table itself doesn't list is the degrees of freedom used in the $t$-test, which is always $N-K-1$ and is listed in the table above as **Df Residuals**.

The value of $df = 97$ is equal to $N-K-1$, so that's what we use for our $t$-tests. ($N$ is the number of observations; $K$ is the number of predictor variables in the regression model)

**Summarize the entire table**: 

The model performs significantly better than you'd expect by chance ($F(2,97) = 215.2$, $p<.001$), which isn't all that surprising: the $R^2 = .812$ value indicates that the regression model accounts for 81.2\% of the variability in the outcome measure. 

However, when we look up at the $t$-tests for each of the individual coefficients, we have pretty strong evidence that the `babysleep` variable has no significant effect; all the work is being done by the `sleep` variable. Taken together, these results suggest that this model is actually the wrong model for the data: you'd probably be better off dropping the `babysleep` predictor entirely. In other words, the simple linear model that we started with is the better model.

In [ ]:
sns.relplot(x='parent_sleep', y='parent_grumpy', data=parenthood_df)

### Calculating standardised regression coefficients

One more thing that you might want to do is to calculate "standardised" regression coefficients, often denoted $\beta$. 

The rationale behind standardised coefficients goes like this: In a lot of situations, your variables are on fundamentally different scales. 

Suppose, for example, my regression model aims to predict people's IQ scores, using their educational attainment (number of years of education) and their income as predictors. 

Obviously, educational attainment and income are not on the same scales: the number of years of schooling can only vary by 10s of years, whereas income could vary by 10,000s of dollars (or more). The units of measurement have a big influence on the regression coefficients: the $b$ coefficients only make sense when interpreted in light of the units, both of the predictor variables and the outcome variable. 

This makes it very difficult to compare the coefficients of different predictors. Yet there are situations where you really do want to make comparisons between different coefficients. This is what **_standardised coefficients_** aim to do.

The basic idea is quite simple: the standardised coefficients are the coefficients that you would have obtained if you'd converted all the variables to $z$-scores before running the regression.  

By converting all the predictors to $z$-scores, they all go into the regression on the same scale, thereby removing the problem of having variables on different scales. Regardless of what the original variables were, a $\beta$ value of 1 means that an increase in the predictor of 1 standard deviation will produce a corresponding 1 standard deviation increase in the outcome variable. Therefore, if variable A has a larger absolute value of $\beta$ than variable B, it is deemed to have a stronger relationship with the outcome. 

Leaving aside the interpretation issues, let's look at how it's calculated. What you could do is standardise all the variables yourself and then run a regression (that's what we do below), but there's anotherway to do it. As it turns out, the $\beta$ coefficient for a predictor $X$ and outcome $Y$ has a very simple formula, namely

$
\beta_X = b_X \times \frac{\sigma_X}{\sigma_Y} 
$

where $\sigma_X$ is the standard deviation of the predictor, and $\sigma_Y$ is the standard deviation of the outcome variable $Y$. 

Let's do it the way where you standardized the data before you run the regression:

#### Z-scoring data

First, let's do something we've seen before which is use the dataframe .apply() method or function. We give this function a an instruction or calcuation that we want to apply to each column of our data.

To get the zscore for each column we will use the zscore() function from the scipy.stats library we previously imported as stats (import scipy.stats as stats)

In [ ]:
standardized_parenthood_df = parenthood_df.apply(stats.zscore)
standardized_parenthood_df.describe()

In [ ]:
mlr = smf.ols(formula="parent_grumpy ~ parent_sleep + baby_sleep", 
              data=standardized_parenthood_df).fit()
mlr.summary()

This clearly shows that the `sleep` variable has a much stronger effect than the `babysleep` variable.

## Assumptions of regression


- *Normality*. Like half the models in statistics, standard linear regression relies on an assumption of normality. Specifically, it assumes that the *residuals* are normally distributed. It's actually okay if the predictors $X$ and the outcome $Y$ are non-normal, so long as the residuals $\epsilon$ are normal.
- *Linearity*. A pretty fundamental assumption of the linear regression model is that relationship between $X$ and $Y$ actually be linear! Regardless of whether it's a simple regression or a multiple regression, we assume that the relatiships involved are linear.
- *Homogeneity of variance*. Strictly speaking, the regression model assumes that each residual $\epsilon_i$ is generated from a normal distribution with mean 0, and (more importantly for the current purposes) with a standard deviation $\sigma$ that is the same for every single residual. In practice, it's impossible to test the assumption that every residual is identically distributed. Instead, what we care about is that the standard deviation of the residual is the same for all values of $\hat{Y}$, and (if we're being especially paranoid) all values of every predictor $X$ in the model. 
- *Uncorrelated predictors*. The idea here is that, is a multiple regression model, you don't want your predictors to be too strongly correlated with each other. This isn't  "technically" an assumption of the regression model, but in practice it's required. Predictors that are too strongly correlated with each other (referred to as "collinearity") can cause problems when evaluating the model. 
- *Residuals are independent of each other*. This is really just a "catch all" assumption, to the effect that "there's nothing else funny going on in the residuals". If there is something weird (e.g., the residuals all depend heavily on some other unmeasured variable) going on, it might screw things up.
- *No "bad" outliers*. Again, not actually a technical assumption of the model (or rather, it's sort of implied by all the others), but there is an implicit assumption that your regression model isn't being too strongly influenced by one or two anomalous data points; since this raises questions about the adequacy of the model, and the trustworthiness of the data in some cases.

We won't go through all of these criteria here, but we will focus on checking the normality of the residuals and verifying that the predictor variables are uncorrelated.


### Checking the normality of the residuals

Like many statistical tools, regression models rely on a normality assumption. In this case, we assume that the residuals are normally distributed. Firstly, it might be good to draw a histogram visualizing the distribution of the residuals.

First we can extract the residuals from our model fit object. In this case we'll use `mlr` from the regression model using parent_sleep and baby_sleep:

In [ ]:
mlr = smf.ols(formula="parent_grumpy ~ parent_sleep + baby_sleep", 
              data=standardized_parenthood_df).fit()

# get the residuals from the model fit object
mlr_resid = mlr.resid
print(mlr_resid)

#This is the error, or distance, of each true y value from our best prediction from the regression line

#### We already now how to make histograms using sns.distplot() like in this example

In [ ]:
sns.displot(mlr_resid)
plt.xlabel("Value of residual")


The plot looks pretty close to normally distributed. We also saw how to formally check the normality of a set of numbers using the Shapiro-Wilk test called shapiro() implemented in the scipy.stats library

In [ ]:
import scipy.stats as stats

W, p = stats.shapiro(x=mlr_resid)


print(W)
print(p)

print('Use numpy round() function to round W and p to three places:')
print(np.round(W,3))
print(np.round(p,3))


The Shapiro-Wilk test is for the null hypothesis that the data are normally distributed. With a test stastic of .99 ($W$) and $p=.84$, we fail to reject the null hypothesis so we conclude that we have (close to) normally distributed data.

#### Checking whether predictors are correlated

To ensure that our predictors are not correlated we can... do a correlation test. We have previously seen how to do this using pingouin (stats_correlation_intro.ipynb)